In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [32]:
import numpy as np

# DNA mapping scheme: binary representation
DNA_MAP = {0: "00", 1: "01", 2: "10", 3: "11"}
REVERSE_DNA_MAP = {v: k for k, v in DNA_MAP.items()}

def dna_encode(image, scheme=DNA_MAP):
    """
    Encodes an image into a DNA string representation using the given scheme.

    Args:
        image (np.ndarray): Input grayscale image as a 2D numpy array.
        scheme (dict): A mapping of pixel mod values to DNA patterns.

    Returns:
        str: Encoded DNA string.
    """
    # Ensure input is a numpy array
    if not isinstance(image, np.ndarray):
        raise ValueError("Input image must be a numpy array.")
    
    # Flatten the image and calculate pixel mod values
    flat_image = image.flatten() % 4
    dna_encoded = ''.join([scheme[val] for val in flat_image])  # Map each pixel to DNA pattern
    
    # Apply XOR operation with a key (for simplicity, using a fixed key here)
    key = 0b1010  # Example key
    dna_encoded_xor = ''.join([format(int(dna_encoded[i:i+2], 2) ^ key, '02b') for i in range(0, len(dna_encoded), 2)])
    
    return dna_encoded_xor

def dna_decode(dna_string, shape, reverse_scheme=REVERSE_DNA_MAP):
    """
    Decodes a DNA string back into the original image using the given scheme.

    Args:
        dna_string (str): DNA string representation of the image.
        shape (tuple): Shape of the original image (rows, cols).
        reverse_scheme (dict): A mapping of DNA patterns back to pixel mod values.

    Returns:
        np.ndarray: Decoded image as a numpy array with the given shape.
    """
    # Ensure the DNA string length is valid
    if len(dna_string) % 2 != 0:
        raise ValueError("DNA string length must be even.")
    
    # Apply XOR operation with the same key used in encoding
    key = 0b1010  # Example key
    dna_decoded_xor = ''.join([format(int(dna_string[i:i+2], 2) ^ key, '02b') for i in range(0, len(dna_string), 2)])
    
    # Decode DNA string back to pixel mod values
    pixels = [reverse_scheme[dna_decoded_xor[i:i+2]] for i in range(0, len(dna_decoded_xor), 2)]
    
    # Reshape into the original shape
    decoded_image = np.array(pixels).reshape(shape)
    
    return decoded_image

In [33]:
# Function to compute image characteristics for dynamic chaotic parameters
def compute_image_characteristics(image_array):
    avg_intensity = np.mean(image_array)  # Average pixel intensity
    variance = np.var(image_array)  # Variance of pixel intensity
    histogram, _ = np.histogram(image_array, bins=256, range=(0, 255))
    probabilities = histogram / np.sum(histogram)
    entropy = -np.sum(p * np.log2(p) for p in probabilities if p > 0)  # Entropy calculation
    return avg_intensity, variance, entropy

# Function to generate Chen attractor keys with dynamic parameters
def generate_chen_keys(size, avg_intensity, variance):
    x, y, z = avg_intensity / 255, variance / 255, 0.1  # Dynamic initialization
    a, b, c = 35 + avg_intensity % 10, 3 + variance % 5, 28  # Adjusted parameters
    keys = []
    for _ in range(size):
        dx = a * (y - x)
        dy = x * (c - z) - y
        dz = x * y - b * z
        x, y, z = x + dx * 0.01, y + dy * 0.01, z + dz * 0.01
        keys.append(x)
    return np.abs(np.array(keys)) % 256  # Normalize keys to range [0, 255]

# Function to generate Lorenz attractor keys with dynamic parameters
def generate_lorenz_keys(size, avg_intensity, entropy):
    x, y, z = avg_intensity / 255, 0.5, entropy / 10  # Dynamic initialization
    sigma, rho, beta = 10 + entropy % 5, 28 + avg_intensity % 10, 2.667
    keys = []
    for _ in range(size):
        dx = sigma * (y - x)
        dy = x * (rho - z) - y
        dz = x * y - beta * z
        x, y, z = x + dx * 0.01, y + dy * 0.01, z + dz * 0.01
        keys.append(y)
    return np.abs(np.array(keys)) % 256  # Normalize keys to range [0, 255]

# Function to generate Logistic Map keys with dynamic parameters
def generate_logistic_keys(size, variance, entropy):
    x = np.clip(variance / 255, 0.01, 0.99)  # Ensure x is in (0, 1)
    r = np.clip(3.99 - (entropy % 0.1), 3.57, 4.0)  # Ensure r is in (3.57, 4.0)
    keys = []
    for _ in range(size):
        x = r * x * (1 - x)  # Logistic map equation
        keys.append(x)
    return (np.array(keys) * 255).astype(np.uint8)


# Function to generate a dynamic S-box
def generate_dynamic_sbox(size, hybrid_keys):
    chaotic_sequence = hybrid_keys[:size]
    return np.argsort(chaotic_sequence)

# DNA Encoding and Decoding
# def dna_encode(image):
#     flat_image = image.flatten()
#     dna_encoded = ''.join([f"{pixel:08b}" for pixel in flat_image])  # Convert each pixel to binary (8 bits)
#     return dna_encoded

# def dna_decode(dna_string, shape):
#     decoded = np.array([int(dna_string[i:i + 8], 2) for i in range(0, len(dna_string), 8)]) 
#     return decoded.reshape(shape)

# Encryption Function
def encrypt_image_enhanced(image, hybrid_keys, sbox, rounds=3):
    flat_image = image.flatten()
    permutation_indices = None
    for _ in range(rounds):
        # Multi-round permutation
        permutation_indices = np.argsort(hybrid_keys[:len(flat_image)])
        permuted_image = flat_image[permutation_indices]
        # Substitution with S-box
        substituted_image = sbox[permuted_image]
        flat_image = substituted_image
    dna_encoded = dna_encode(flat_image.reshape(image.shape))  # DNA encoding
    return flat_image.reshape(image.shape), dna_encoded, permutation_indices

# Decryption Function
def decrypt_image_enhanced(encrypted_dna, permutation_indices, hybrid_keys, sbox_inv, shape, rounds=3):
    flat_image = dna_decode(encrypted_dna, shape).flatten()
    for _ in range(rounds):
        # Reverse substitution
        substituted_image = np.array([sbox_inv[pixel] for pixel in flat_image])
        # Reverse permutation
        reverse_indices = np.argsort(permutation_indices)
        flat_image = substituted_image[reverse_indices]
    return flat_image.reshape(shape)

# Main function
def main(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("L")
    image_array = np.array(image)

    # Compute dynamic characteristics
    avg_intensity, variance, entropy = compute_image_characteristics(image_array)

    # Generate chaotic keys
    chen_keys = generate_chen_keys(image_array.size, avg_intensity, variance)
    lorenz_keys = generate_lorenz_keys(image_array.size, avg_intensity, entropy)
    logistic_keys = generate_logistic_keys(image_array.size, variance, entropy)
    hybrid_keys = (chen_keys + lorenz_keys + logistic_keys) % 256

    # Generate S-box and its inverse
    sbox = generate_dynamic_sbox(256, hybrid_keys)
    sbox_inv = np.argsort(sbox)

    # Encrypt the image
    encrypted_image, encrypted_dna, permutation_indices = encrypt_image_enhanced(
        image_array, hybrid_keys, sbox
    )

    # Decrypt the image
    decrypted_image = decrypt_image_enhanced(
        encrypted_dna, permutation_indices, hybrid_keys, sbox_inv, image_array.shape
    )

    # Display results


In [17]:
# %pip install opencv-python
# %pip install pydub
%pip install SpeechRecognition

   ---------------------------------------- 0.0/32.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/32.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/32.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/32.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/32.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/32.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/32.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/32.8 MB ? eta -:--:--
    --------------------------------------- 0.5/32.8 MB 220.7 kB/s eta 0:02:27
    --------------------------------------- 0.5/32.8 MB 220.7 kB/s eta 0:02:27
    --------------------------------------- 0.5/32.8 MB 220.7 kB/s eta 0:02:27
    --------------------------------------- 0.5/32.8 MB 220.7 kB/s eta 0:02:27
    --------------------------------------- 0.5/32.8 MB 220.7 kB/s eta 0:02:27
    --------------------------------------- 0


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [43]:
def calculate_npcr_uaci(original_img_array, encrypted_img_array):
    # Ensure the images are of the same size
    # assert original_img_array.shape == encrypted_img_array.shape, "Images must have the same dimensions"
    
    # Calculate NPCR
    diff = original_img_array != encrypted_img_array
    npcr = np.sum(diff) / original_img_array.size * 100
    
    diff = np.abs(original_img_array - encrypted_img_array).astype(np.float32)
    uaci = np.sum(diff) / (255 * original_img_array.size) * 100
    # Calculate UACI
    # uaci = np.sum(np.abs(original_img_array - encrypted_img_array)) / (original_img_array.size * 255) * 100
    
    print("npcr :",npcr ," ","uaci :",uaci)
    return npcr,uaci

In [45]:
import numpy as np
from PIL import Image


def compute_image_characteristics(image_array):
    avg_intensity = np.mean(image_array)  # Average pixel intensity
    variance = np.var(image_array)  # Variance of pixel intensity
    histogram, _ = np.histogram(image_array, bins=256, range=(0, 255))
    probabilities = histogram / np.sum(histogram)
    entropy = -np.sum(p * np.log2(p) for p in probabilities if p > 0)  # Entropy calculation
    return avg_intensity, variance, entropy


def generate_chen_keys(size, avg_intensity, variance):
    x, y, z = avg_intensity / 255, variance / 255, 0.1  # Dynamic initialization
    a, b, c = 35 + avg_intensity % 10, 3 + variance % 5, 28  # Adjusted parameters
    keys = []
    for _ in range(size):
        dx = a * (y - x)
        dy = x * (c - z) - y
        dz = x * y - b * z
        x, y, z = x + dx * 0.01, y + dy * 0.01, z + dz * 0.01
        keys.append(x)
    return np.abs(np.array(keys)) % 256  # Normalize keys to range [0, 255]

# Function to generate Lorenz attractor keys with dynamic parameters
def generate_lorenz_keys(size, avg_intensity, entropy):
    x, y, z = avg_intensity / 255, 0.5, entropy / 10  # Dynamic initialization
    sigma, rho, beta = 10 + entropy % 5, 28 + avg_intensity % 10, 2.667
    keys = []
    for _ in range(size):
        dx = sigma * (y - x)
        dy = x * (rho - z) - y
        dz = x * y - beta * z
        x, y, z = x + dx * 0.01, y + dy * 0.01, z + dz * 0.01
        keys.append(y)
    return np.abs(np.array(keys)) % 256  # Normalize keys to range [0, 255]

# Function to generate Logistic Map keys with dynamic parameters
def generate_logistic_keys(size, variance, entropy):
    x = np.clip(variance / 255, 0.01, 0.99)  # Ensure x is in (0, 1)
    r = np.clip(3.99 - (entropy % 0.1), 3.57, 4.0)  # Ensure r is in (3.57, 4.0)
    keys = []
    for _ in range(size):
        x = r * x * (1 - x)  # Logistic map equation
        keys.append(x)
    return (np.array(keys) * 255).astype(np.uint8)

# Function to generate a dynamic S-box
def generate_dynamic_sbox(size, hybrid_keys):
    chaotic_sequence = hybrid_keys[:size]
    return np.argsort(chaotic_sequence)

# DNA Encoding and Decoding
def dna_encode(image):
    flat_image = image.flatten()
    dna_encoded = ''.join([f"{pixel:08b}" for pixel in flat_image])  # Convert each pixel to binary (8 bits)
    return dna_encoded

def dna_decode(dna_string, shape):
    decoded = np.array([int(dna_string[i:i + 8], 2) for i in range(0, len(dna_string), 8)]) 
    return decoded.reshape(shape)

# Encryption Function with Chen keys first
def encrypt_with_chen(image_array, chen_keys):
    flat_image = image_array.flatten()
    permutation_indices_chen = np.argsort(chen_keys[:len(flat_image)])
    permuted_image_chen = flat_image[permutation_indices_chen]
    return permuted_image_chen.reshape(image_array.shape), permutation_indices_chen

# Encryption Function with Lorenz keys second
def encrypt_with_lorenz(image_array, lorenz_keys):
    flat_image = image_array.flatten()
    permutation_indices_lorenz = np.argsort(lorenz_keys[:len(flat_image)])
    permuted_image_lorenz = flat_image[permutation_indices_lorenz]
    return permuted_image_lorenz.reshape(image_array.shape), permutation_indices_lorenz

# Encryption Function with Logistic keys third
def encrypt_with_logistic(image_array, logistic_keys):
    flat_image = image_array.flatten()
    permutation_indices_logistic = np.argsort(logistic_keys[:len(flat_image)])
    permuted_image_logistic = flat_image[permutation_indices_logistic]
    return permuted_image_logistic.reshape(image_array.shape), permutation_indices_logistic

# Final Encryption Function with S-box and hybrid keys
def encrypt_with_sbox(image_array, hybrid_keys):
    flat_image = image_array.flatten()
    sbox = generate_dynamic_sbox(256, hybrid_keys)
    substituted_image_sbox = sbox[flat_image]
    dna_encoded_sbox = dna_encode(substituted_image_sbox.reshape(image_array.shape))
    return substituted_image_sbox.reshape(image_array.shape), dna_encoded_sbox

# Decryption Functions for each step in reverse order
def decrypt_with_sbox(encrypted_dna_sbox, hybrid_keys, shape):
    sbox_inv = np.argsort(generate_dynamic_sbox(256, hybrid_keys))
    substituted_image_sbox_inv_flattened = dna_decode(encrypted_dna_sbox, shape).flatten()
    substituted_image_sbox_inv = sbox_inv[substituted_image_sbox_inv_flattened]
    return substituted_image_sbox_inv.reshape(shape)

def decrypt_with_logistic(permuted_image_logistic_flattened, logistic_keys, shape):
    reverse_indices_logistic = np.argsort(np.argsort(logistic_keys[:len(permuted_image_logistic_flattened)]))
    permuted_image_logistic = permuted_image_logistic_flattened[reverse_indices_logistic]
    return permuted_image_logistic.reshape(shape)

def decrypt_with_lorenz(permuted_image_lorenz_flattened, lorenz_keys, shape):
    reverse_indices_lorenz = np.argsort(np.argsort(lorenz_keys[:len(permuted_image_lorenz_flattened)]))
    permuted_image_lorenz = permuted_image_lorenz_flattened[reverse_indices_lorenz]
    return permuted_image_lorenz.reshape(shape)

def decrypt_with_chen(permuted_image_chen_flattened, chen_keys, shape):
    reverse_indices_chen = np.argsort(np.argsort(chen_keys[:len(permuted_image_chen_flattened)]))
    permuted_image_chen = permuted_image_chen_flattened[reverse_indices_chen]
    return permuted_image_chen.reshape(shape)

def main(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("L")
    image_array = np.array(image)

    # Compute dynamic characteristics
    avg_intensity, variance, entropy = compute_image_characteristics(image_array)

    # Generate chaotic keys
    chen_keys = generate_chen_keys(image_array.size, avg_intensity, variance)
    lorenz_keys = generate_lorenz_keys(image_array.size, avg_intensity, entropy)
    logistic_keys = generate_logistic_keys(image_array.size, variance, entropy)
    hybrid_keys = (chen_keys + lorenz_keys + logistic_keys) % 256

    # Encrypt the image step by step
    encrypted_image_chen, pe = encrypt_with_chen(image_array, chen_keys)
    


    encrypted_image_lorenz, p = encrypt_with_lorenz(encrypted_image_chen, lorenz_keys)
  
 

    encrypted_image_logistic, p = encrypt_with_logistic(encrypted_image_lorenz, logistic_keys)


    encrypted_image_sbox, encrypted_dna_sbox = encrypt_with_sbox(encrypted_image_logistic, hybrid_keys)
    # npcr_sbox, uaci_sbox = calculate_npcr_uaci( image_array,encrypt_with_lorenz)
    # print(npcr_sbox,uaci_sbox)

    # Decrypt the image step by step
    decrypted_image_logistic = decrypt_with_sbox(encrypted_dna_sbox, hybrid_keys, encrypted_image_logistic.shape)
    decrypted_image_lorenz = decrypt_with_logistic(decrypted_image_logistic.flatten(), logistic_keys, encrypted_image_lorenz.shape)
    decrypted_image_chen = decrypt_with_lorenz(decrypted_image_lorenz.flatten(), lorenz_keys, encrypted_image_chen.shape)
    decrypted_image = decrypt_with_chen(decrypted_image_chen.flatten(), chen_keys, image_array.shape)

    # Ensure the arrays are of type uint8 for image display
    encrypted_image_sbox = encrypted_image_sbox.astype(np.uint8)
    decrypted_image = decrypted_image.astype(np.uint8)

    # Display results
    Image.fromarray(encrypted_image_sbox).show(title="Encrypted Image")
    Image.fromarray(decrypted_image).show(title="Decrypted Image")
if __name__ == "__main__":
    main("C:/Users/20092/OneDrive/Documents/Bhumit/cyptography/slider2.webp")

C:\Users\20092\AppData\Local\Temp\ipykernel_50184\2871529253.py:10: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  entropy = -np.sum(p * np.log2(p) for p in probabilities if p > 0)  # Entropy calculation


In [ ]:
import numpy as np
from PIL import Image

# Function to compute image characteristics for dynamic chaotic parameters
def compute_image_characteristics(image_array):
    avg_intensity = np.mean(image_array)  # Average pixel intensity
    variance = np.var(image_array)  # Variance of pixel intensity
    histogram, _ = np.histogram(image_array, bins=256, range=(0, 255))
    probabilities = histogram / np.sum(histogram)
    entropy = -np.sum(p * np.log2(p) for p in probabilities if p > 0)  # Entropy calculation
    return avg_intensity, variance, entropy

# Function to generate Rossler attractor keys with dynamic parameters
def generate_rossler_keys(size, avg_intensity, variance):
    x, y, z = avg_intensity / 255, variance / 255, 0.1
    a, b, c = 0.2, 0.2, 5.7
    keys = []
    for _ in range(size):
        dx = -y - z
        dy = x + a * y
        dz = b + z * (x - c)
        x, y, z = x + dx * 0.01, y + dy * 0.01, z + dz * 0.01
        keys.append(x)
    return np.abs(np.array(keys)) % 256

# Function to generate Henon map keys with dynamic parameters
def generate_henon_keys(size, avg_intensity, entropy):
    x, y = avg_intensity / 255, entropy / 10
    a, b = 1.4, 0.3
    keys = []
    for _ in range(size):
        x, y = 1 - a * x**2 + y, b * x
        keys.append(x)
    return np.abs(np.array(keys)) % 256

# Function to generate a dynamic S-box
def generate_dynamic_sbox(size, hybrid_keys):
    chaotic_sequence = hybrid_keys[:size]
    return np.argsort(chaotic_sequence)

def generate_rossler_keys(size, avg_intensity, variance):
    x, y, z = avg_intensity / 255, variance / 255, 0.1
    a, b, c = 0.2, 0.2, 5.7
    keys = []
    for _ in range(size):
        dx = -y - z
        dy = x + a * y
        dz = b + z * (x - c)
        x, y, z = x + dx * 0.01, y + dy * 0.01, z + dz * 0.01
        keys.append(x)
    return np.abs(np.array(keys)) % 256

# Function to generate Henon map keys with dynamic parameters
def generate_henon_keys(size, avg_intensity, entropy):
    x, y = avg_intensity / 255, entropy / 10
    a, b = 1.4, 0.3
    keys = []
    for _ in range(size):
        x, y = 1 - a * x**2 + y, b * x
        keys.append(x)
    return np.abs(np.array(keys)) % 256

# Encryption Function with Rossler keys
def encrypt_with_rossler(image_array, rossler_keys):
    flat_image = image_array.flatten()
    permutation_indices_rossler = np.argsort(rossler_keys[:len(flat_image)])
    permuted_image_rossler = flat_image[permutation_indices_rossler]
    return permuted_image_rossler.reshape(image_array.shape), permutation_indices_rossler
# Encryption Function with Henon keys
def encrypt_with_henon(image_array, henon_keys):
    flat_image = image_array.flatten()
    permutation_indices_henon = np.argsort(henon_keys[:len(flat_image)])
    permuted_image_henon = flat_image[permutation_indices_henon]
    return permuted_image_henon.reshape(image_array.shape), permutation_indices_henon
# DNA Encoding and Decoding
def dna_encode(image):
    flat_image = image.flatten()
    dna_encoded = ''.join([f"{pixel:08b}" for pixel in flat_image])  # Convert each pixel to binary (8 bits)
    return dna_encoded

def dna_decode(dna_string, shape):
    decoded = np.array([int(dna_string[i:i + 8], 2) for i in range(0, len(dna_string), 8)]) 
    return decoded.reshape(shape)

# Encryption Function with S-box and hybrid keys
def encrypt_with_sbox(image_array, hybrid_keys):
    flat_image = image_array.flatten()
    sbox = generate_dynamic_sbox(256, hybrid_keys)
    substituted_image_sbox = sbox[flat_image]
    dna_encoded_sbox = dna_encode(substituted_image_sbox.reshape(image_array.shape))
    return substituted_image_sbox.reshape(image_array.shape), dna_encoded_sbox

# Decryption Functions for each step in reverse order
def decrypt_with_sbox(encrypted_dna_sbox, hybrid_keys, shape):
    sbox_inv = np.argsort(generate_dynamic_sbox(256, hybrid_keys))
    substituted_image_sbox_inv_flattened = dna_decode(encrypted_dna_sbox, shape).flatten()
    substituted_image_sbox_inv = sbox_inv[substituted_image_sbox_inv_flattened]
    return substituted_image_sbox_inv.reshape(shape)

def decrypt_with_henon(permuted_image_henon_flattened, henon_keys, shape):
    reverse_indices_henon = np.argsort(np.argsort(henon_keys[:len(permuted_image_henon_flattened)]))
    permuted_image_henon = permuted_image_henon_flattened[reverse_indices_henon]
    return permuted_image_henon.reshape(shape)

def decrypt_with_rossler(permuted_image_rossler_flattened, rossler_keys, shape):
    reverse_indices_rossler = np.argsort(np.argsort(rossler_keys[:len(permuted_image_rossler_flattened)]))
    permuted_image_rossler = permuted_image_rossler_flattened[reverse_indices_rossler]
    return permuted_image_rossler.reshape(shape)

# Function to calculate NPCR and UACI factors
def calculate_npcr_uaci(original_img_array, encrypted_img_array):
    assert original_img_array.shape == encrypted_img_array.shape, "Images must have the same dimensions"
    diff = original_img_array != encrypted_img_array
    npcr = np.sum(diff) / original_img_array.size * 100
    uaci = np.sum(np.abs(original_img_array - encrypted_img_array)) / (original_img_array.size * 255) * 100
    return npcr, uaci

# Placeholder function for Quantum Key Distribution (QKD)
def quantum_key_distribution():
    # This is a placeholder function. In a real-world scenario,
    # QKD would involve complex quantum communication protocols.
    # Here we simulate QKD by generating a random key.
    key_length = 256 // 8  # 256-bit key
    qkd_key = np.random.randint(0, 256, key_length).astype(np.uint8)
    return qkd_key

# Main function to encrypt and decrypt the image using the enhanced algorithm
def main(image_path):
    image = Image.open(image_path).convert("L")
    image_array = np.array(image)

    avg_intensity, variance, entropy = compute_image_characteristics(image_array)
    
    # Generate QKD key
    qkd_key = quantum_key_distribution()

    rossler_keys = generate_rossler_keys(image_array.size, avg_intensity, variance)
    henon_keys = generate_henon_keys(image_array.size, avg_intensity, entropy)
    
    # Combine chaotic keys with QKD key for hybrid encryption
    hybrid_keys = (rossler_keys + henon_keys + qkd_key) % 256

    encrypted_image_rossler, permutation_indices_rossler = encrypt_with_rossler(image_array, rossler_keys)
    npcr_rossler, uaci_rossler = calculate_npcr_uaci(image_array, encrypted_image_rossler)
    print(f"NPCR after Rossler encryption: {npcr_rossler:.2f}%")
    print(f"UACI after Rossler encryption: {uaci_rossler:.2f}%")

    encrypted_image_henon, permutation_indices_henon = encrypt_with_henon(encrypted_image_rossler.flatten(), henon_keys)
    npcr_henon, uaci_henon = calculate_npcr_uaci(encrypted_image_rossler, encrypted_image_henon)
    print(f"NPCR after Henon encryption: {npcr_henon:.2f}%")
    print(f"UACI after Henon encryption: {uaci_henon:.2f}%")

    encrypted_image_sbox, encrypted_dna_sbox = encrypt_with_sbox(encrypted_image_henon, hybrid_keys)
    npcr_sbox, uaci_sbox = calculate_npcr_uaci(encrypted_image_henon, encrypted_image_sbox)
    print(f"NPCR after S-box encryption: {npcr_sbox:.2f}%")
    print(f"UACI after S-box encryption: {uaci_sbox:.2f}%")

    # Decrypt the image step by step
    decrypted_image_henon = decrypt_with_sbox(encrypted_dna_sbox, hybrid_keys, encrypted_image_henon.shape)
    decrypted_image_rossler = decrypt_with_henon(decrypted_image_henon.flatten(), henon_keys, encrypted_image_rossler.shape)
    decrypted_image = decrypt_with_rossler(decrypted_image_rossler.flatten(), rossler_keys, image_array.shape)

    # Ensure the arrays are of type uint8 for image display
    encrypted_image_sbox = encrypted_image_sbox.astype(np.uint8)
    decrypted_image = decrypted_image.astype(np.uint8)

    # Display results
    Image.fromarray(encrypted_image_sbox).show(title="Encrypted Image")
    Image.fromarray(decrypted_image).show(title="Decrypted Image")

if __name__ == "__main__":
    main("C:/Users/20092/OneDrive/Documents/Bhumit/cyptography/slider2.webp")
  
 
   

C:\Users\20092\AppData\Local\Temp\ipykernel_50184\244106571.py:10: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  entropy = -np.sum(p * np.log2(p) for p in probabilities if p > 0)  # Entropy calculation


ValueError: operands could not be broadcast together with shapes (604160,) (32,) 